In [ ]:
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, default_argument_parser, default_setup, hooks, launch

# train data
name        = "TRAIN"
json_file   = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train_mod.json"
image_root  = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/images/livecell_train_val_images/"
# test data
name_val        = "TEST"
json_file_val   = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test_mod.json"
image_root_val  = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/images/livecell_test_images/"
# registr
register_coco_instances(name, {}, json_file, image_root)
register_coco_instances(name_val, {}, json_file_val, image_root_val)


In [ ]:
args = default_argument_parser().parse_args("--config-file /home/ec2-user/detectron2-ResNeSt/configs/LiveCell/model/anchor_based/livecell_config.yaml --eval-only OUTPUT_DIR /home/ec2-user/resnest_output MODEL.WEIGHTS http://livecell-dataset.s3.eu-central-1.amazonaws.com/LIVECell_dataset_2021/models/Anchor_based/ALL/LIVECell_anchor_based_model.pth".split())

In [ ]:
from detectron2.data.datasets import register_coco_instances

# train data
name        = "TRAIN"
json_file   = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_train_mod.json"
image_root  = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/images/livecell_train_val_images/"
# test data
name_val        = "TEST"
json_file_val   = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/annotations/LIVECell/livecell_coco_test_mod.json"
image_root_val  = "/home/ec2-user/livecell-dataset/LIVECell_dataset_2021/images/livecell_test_images/"
# registr
register_coco_instances(name, {}, json_file, image_root)
register_coco_instances(name_val, {}, json_file_val, image_root_val)

In [ ]:
from detectron2.config import get_cfg

cfg = get_cfg()
cfg.merge_from_file('/home/ec2-user/detectron2-ResNeSt/configs/LiveCell/model/anchor_based/livecell_config.yaml')
cfg.merge_from_list(args.opts)
cfg.freeze()
default_setup(cfg, args)

In [ ]:
# From detectron2-ResNeSt main folder or here: https://github.com/bendangnuksung/detectron2-ResNeSt/blob/resnest/tools/train_net.py
class Trainer(DefaultTrainer):
    """
    We use the "DefaultTrainer" which contains pre-defined default logic for
    standard training workflow. They may not work for you, especially if you
    are working on a new research project. In that case you can use the cleaner
    "SimpleTrainer", or write your own training loop. You can use
    "tools/plain_train_net.py" as an example.
    """

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        """
        Create evaluator(s) for a given dataset.
        This uses the special metadata "evaluator_type" associated with each builtin dataset.
        For your own dataset, you can simply create an evaluator manually in your
        script and do not have to worry about the hacky if-else logic here.
        """
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        evaluator_list = []
        evaluator_type = MetadataCatalog.get(dataset_name).evaluator_type
        if evaluator_type in ["sem_seg", "coco_panoptic_seg"]:
            evaluator_list.append(
                SemSegEvaluator(
                    dataset_name,
                    distributed=True,
                    num_classes=cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES,
                    ignore_label=cfg.MODEL.SEM_SEG_HEAD.IGNORE_VALUE,
                    output_dir=output_folder,
                )
            )
        if evaluator_type in ["coco", "coco_panoptic_seg"]:
            evaluator_list.append(COCOEvaluator(dataset_name, cfg, True, output_folder))
        if evaluator_type == "coco_panoptic_seg":
            evaluator_list.append(COCOPanopticEvaluator(dataset_name, output_folder))
        elif evaluator_type == "cityscapes":
            assert (
                torch.cuda.device_count() >= comm.get_rank()
            ), "CityscapesEvaluator currently do not work with multiple machines."
            return CityscapesEvaluator(dataset_name)
        elif evaluator_type == "pascal_voc":
            return PascalVOCDetectionEvaluator(dataset_name)
        elif evaluator_type == "lvis":
            return LVISEvaluator(dataset_name, cfg, True, output_folder)
        if len(evaluator_list) == 0:
            raise NotImplementedError(
                "no Evaluator for the dataset {} with the type {}".format(
                    dataset_name, evaluator_type
                )
            )
        elif len(evaluator_list) == 1:
            return evaluator_list[0]
        return DatasetEvaluators(evaluator_list)

    @classmethod
    def test_with_TTA(cls, cfg, model):
        logger = logging.getLogger("detectron2.trainer")
        # In the end of training, run an evaluation with TTA
        # Only support some R-CNN models.
        logger.info("Running inference with test-time augmentation ...")
        model = GeneralizedRCNNWithTTA(cfg, model)
        evaluators = [
            cls.build_evaluator(
                cfg, name, output_folder=os.path.join(cfg.OUTPUT_DIR, "inference_TTA")
            )
            for name in cfg.DATASETS.TEST
        ]
        res = cls.test(cfg, model, evaluators)
        res = OrderedDict({k + "_TTA": v for k, v in res.items()})
        return res



In [ ]:
model = Trainer.build_model(cfg)
model

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer

DetectionCheckpointer(model, save_dir=cfg.OUTPUT_DIR).resume_or_load(
            cfg.MODEL.WEIGHTS, resume=args.resume
        )

In [ ]:
# Dataloader
dataset_name = cfg.DATASETS.TEST[0]
data_loader = Trainer.build_test_loader(cfg, dataset_name)
print(len(data_loader))

In [ ]:
from detectron2.data import MetadataCatalog
from detectron2.evaluation import COCOEvaluator # This is teh LiveCell COCOEvaluator that is modified to detect more than 100 instances per image

# Build Evaluator
evaluator = Trainer.build_evaluator(cfg, dataset_name)
data_loader = Trainer.build_test_loader(cfg, dataset_name)

In [ ]:
# !pip install opencv-python

In [ ]:
# Slight modifications of the detectron2 visualizer function from here: https://github.com/facebookresearch/detectron2/blob/7801ac3d595a0663d16c0c9a6a339c77b2ddbdfb/detectron2/utils/visualizer.py#L538
import numpy as np
from detectron2.structures import BitMasks, Boxes, BoxMode

def my_draw_dataset_dict(v, annos):
    """
    Draw annotations/segmentaions in Detectron2 Dataset format.

    Args:
        v: Detectron2 Visualizer object
        dic (dict): annotation/segmentation data of one image, in Detectron2 Dataset format.

    Returns:
        output (VisImage): image object with visualizations.
    """

    if annos:
        if "segmentation" in annos[0]:
            masks = [x["segmentation"] for x in annos]
        else:
            masks = None
        if "keypoints" in annos[0]:
            keypts = [x["keypoints"] for x in annos]
            keypts = np.array(keypts).reshape(len(annos), -1, 3)
        else:
            keypts = None

        boxes = [BoxMode.convert(x["bbox"], x["bbox_mode"], BoxMode.XYXY_ABS) for x in annos]

        labels = [x["category_id"] for x in annos]
        colors = None
        if v._instance_mode == ColorMode.SEGMENTATION and v.metadata.get("thing_colors"):
            colors = [
                v._jitter([x / 255 for x in v.metadata.thing_colors[c]]) for c in labels
            ]
        names = v.metadata.get("thing_classes", None)
        if names:
            labels = [names[i] for i in labels]
        labels = [
            "{}".format(i) + ("|crowd" if a.get("iscrowd", 0) else "")
            for i, a in zip(labels, annos)
        ]
        v.overlay_instances(
            labels=labels, boxes=boxes, masks=masks, keypoints=keypts, assigned_colors=colors
        )

    # sem_seg = None
    # if sem_seg is None and "sem_seg_file_name" in dic:
    #     sem_seg = cv2.imread(dic["sem_seg_file_name"], cv2.IMREAD_GRAYSCALE)
    # if sem_seg is not None:
    #     self.draw_sem_seg(sem_seg, area_threshold=0, alpha=0.5)
    return v.output

In [ ]:
import os, json, cv2, random
import matplotlib.pyplot as plt
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import ColorMode

sel_idx = 18
sel_img = evaluator._coco_api.dataset['images'][sel_idx]

# Create annotations for the selected image
annos = evaluator._coco_api.dataset.get("annotations", None)
sel_img_annos = []
for a in annos:
    if a['image_id']==sel_img['id']:
        a['bbox_mode'] = BoxMode.XYWH_ABS
        sel_img_annos.append(a)
len(sel_img_annos)

# Plot image and annotations
im = cv2.imread(image_root_val+ sel_img['file_name'])
v = Visualizer(im[:, :, ::-1],
                   metadata={'thing_classes':['no_cell','cell']},
                   scale=0.5, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
    )


plt.figure(figsize=(50, 100))
plt.subplot(1, 2, 1)
# Raw image
plt.imshow(v.output.get_image()[:, :, ::-1])
# Image with annotations
out = my_draw_dataset_dict(v, sel_img_annos)
plt.subplot(1, 2, 2)
plt.imshow(out.get_image()[:, :, ::-1])
plt.show()

# Investigate Model Outputs from the different modules

In [ ]:
from contextlib import contextmanager

@contextmanager
def inference_context(model):
    """
    A context where the model is temporarily changed to eval mode,
    and restored to previous mode afterwards.

    Args:
        model: a torch Module
    """
    training_mode = model.training
    model.eval()
    yield
    model.train(training_mode)


In [ ]:
evaluator.reset()

In [ ]:
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
import torch

with inference_context(model), torch.no_grad():
    for idx, inputs in enumerate(data_loader):
        if inputs[0]['image_id']==sel_img['id']:
            images = model.preprocess_image(inputs)
            features = model.backbone(images.tensor)        
            outputs = model(inputs)
            break

## Backbone

In [ ]:
from detectron2.layers import (
    Conv2d,
    DeformConv,
    FrozenBatchNorm2d,
    ModulatedDeformConv,
    ShapeSpec,
    get_norm,
)

import torch.nn.functional as F

### Stem

In [ ]:
with inference_context(model), torch.no_grad():
    bu_stem = model.backbone.bottom_up.stem(images.tensor)
    print(images.tensor.shape)
    # Stem: First convolutions
    print(bu_stem.shape)

    # Architecture of Stem
    model.backbone.bottom_up.stem.deep_stem
    print(model.backbone.bottom_up.stem.conv1_1)
    print(model.backbone.bottom_up.stem.conv1_1.weight.shape)
    stem_out_1 = model.backbone.bottom_up.stem.conv1_1(images.tensor)
    print(stem_out_1.shape)
    stem_out_1 = F.relu_(stem_out_1)
    print(stem_out_1.shape)
    print(model.backbone.bottom_up.stem.conv1_2.weight.shape)
    stem_out_2 = model.backbone.bottom_up.stem.conv1_2(stem_out_1)
    stem_out_2 = F.relu_(stem_out_2)
    print(stem_out_2.shape)
    print(model.backbone.bottom_up.stem.conv1_3.weight.shape)
    stem_out_3 = model.backbone.bottom_up.stem.conv1_3(stem_out_1)
    stem_out_3 = F.relu_(stem_out_3)
    print(stem_out_3.shape)
    # Final maxpooling for the stem block
    stem_out = F.max_pool2d(stem_out_3, kernel_size=3, stride=2, padding=1)
    print(stem_out.shape)

    bu_stem = model.backbone.bottom_up.stem(images.tensor)
    print(bu_stem.shape)

### Res2

In [ ]:
# Number of Blocks in this layer
with inference_context(model), torch.no_grad():
    print(len(model.backbone.bottom_up.res2))
    model.backbone.bottom_up.res2[0]
    print(model.backbone.bottom_up.res2(bu_stem).shape)

In [ ]:
model.backbone.bottom_up.res2[0]

In [ ]:
model.backbone.bottom_up.res2

In [ ]:
# model.backbone.bottom_up._out_features
type(model.backbone.bottom_up)
print(model.backbone.bottom_up._out_features)
for stage,name in model.backbone.bottom_up.stages_and_names:
    print(stage)
    print(name)
    break

In [ ]:
with inference_context(model), torch.no_grad():
    bu = model.backbone.bottom_up(images.tensor)
    bu_stem = model.backbone.bottom_up.stem(images.tensor)

In [ ]:
cfg.MODEL.RESNETS.STEM_OUT_CHANNELS
print(model.backbone.bottom_up._out_feature_strides)
print(model.backbone.bottom_up._out_feature_channels)

In [ ]:

model.backbone.bottom_up.stages_and_names

In [ ]:
print(images.tensor.shape)
# Stem: First convolutions
print(bu_stem.shape)

# Full Backbone
print(bu['res2'].shape)
print(bu['res3'].shape)
print(bu['res4'].shape)
print(bu['res5'].shape)

In [ ]:
print(len(outputs[0]['instances']))
print(len(sel_img_annos))
outputs[0]['instances'][0]